Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Инструкция по выполнению проекта
1. Загрузите и изучите данные.
2. Ответьте на вопрос и обоснуйте решение.
- Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
    - 1. Изменится. Приведите примеры матриц.
    - 2. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
3. Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.
4. Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression as lr
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error

In [2]:
insurance = pd.read_csv('/datasets/insurance.csv')
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


*Пропусков нет*

In [3]:
display(insurance.head())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
# Проверим на дубликаты
print(f"Кол-во дубликатов: {insurance.duplicated().sum()}")

Кол-во дубликатов: 153


In [5]:
# Удалим дубликаты
insurance = insurance.drop_duplicates().reset_index(drop=True)

## Вывод по 1 шагу

Данные изучены, были выявлены дубликаты и после удалены

## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится

**Обоснование:**

*Для доказательства подставим вместо $X$ подставим $XZ$*:

$$
w_{new} = ((XZ)^TZX)^{-1}(XZ)^Ty
$$

подставим $w_{new}$ в $a_{new}$ и преобразуем:

$a_{new} = XZ((XZ)^TZX)^{-1}(XZ)^Ty$

$a_{new} = XZ(Z^TX^TXZ)^{-1}Z^TX^Ty$

$a_{new} = XZ(X^TXZ)^{-1}Z^TX^Ty$

$a_{new} = XZ(X^TXZ)^{-1}(Z^T)^{-1}Z^TX^Ty$

$a_{new} = XZZ^{-1}(X^TX)^{-1}(Z^T)^{-1}Z^TX^Ty$

$a_{new} = XE(X^TX)^{-1}EX^Ty$

$a_{new} = X(X^TX)^{-1}X^Ty = Xw$

**Рассуждение:**
1. Раскрываем транспонирования по свойству: $(AB)^T = B^T A^T$
2. Считаем $X^TX$ - квадратная обратимая марица, $Z$ - обратимая по опеределнию, то раскрываем скобки по свойству $(AB)^{-1} = B^{-1} A^{-1}$ 2 раза
3. Используя ассоциативность умножения матриц и определение обратной матрицы получим ответ.

Таким образом получаем что:
$$
a_{new} = a
$$

## 3. Алгоритм преобразования

**Алгоритм**

1. Создаём квадратную матрицу A из рандомных чисел с помощью np.random.randint(): её стороны равны количеству признаков датафрейма, подлежащего кодировке.
2. Проверяем созданную матрицу на обратимость с помощью np.linalg.inv(). Если она обратима, получаем матрицу $A^{-1}$. Если нет, повторяем генерацию матрицы.
3. Умножаем матрицу признаков на полученную квадратную матрицу и получаем закодированные данные.
4. Для раскодирования используем умножение матрицы закодированных параметров на матрицу $A^{-1}$
5. Проверяем качество раскодирования.
6. Устраняем выявленные нарушения работы алгоритма.

**Обоснование**

*Умножение матрицы на обратимую квадратную матрицу изменит данные датафрейма, что защитит данные клиента. Также в наших руках будет ключ к дешифровке этой информации - обратная матрица. Данный подход не должен ухудшить качество модели линейной регрессии.*

## 4. Проверка алгоритма

In [6]:
# Разделим на признаки и целевые признаки
target = insurance['Страховые выплаты']
features = insurance.drop(['Страховые выплаты'], axis=1)

# Разделим обучающую и валидационную выборки
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

# Преобразуем признаки в матрицы:
matrix_features_train = features_train.values
matrix_features_valid = features_valid.values

*Сгенерируем матрицу и проверим ее на обратимость*

In [7]:
random_matrix = np.random.RandomState(12345).randint(0, 100, size = (4, 4))
random_matrix

array([[98, 29,  1, 36],
       [41, 34, 29,  1],
       [59, 14, 91, 80],
       [73, 11, 77, 10]])

In [8]:
np.round(random_matrix.dot(np.linalg.inv(random_matrix)))

array([[ 1., -0., -0.,  0.],
       [ 0.,  1., -0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0., -0.,  0.,  1.]])

*Получилась единичная матрица - значит, сгенерированная матрица обратима.*

In [9]:
# Зашифруем данные перемножив признаки на сгенерированную матрицу:
encrypted_matrix_features_train = np.dot(matrix_features_train, random_matrix)
encrypted_matrix_features_valid = np.dot(matrix_features_valid, random_matrix)

Обучим и посмотрим на метрику R2 модели:

In [10]:
model = lr()
model.fit(matrix_features_train, target_train)
model_predicted = model.predict(matrix_features_valid)

In [11]:
encoded_model = lr()
encoded_model.fit(encrypted_matrix_features_train, target_train)
encoded_models_predicted = encoded_model.predict(encrypted_matrix_features_valid)

In [12]:
r2score = r2_score(target_valid, model_predicted)
encoded_models_r2_score = r2_score(target_valid, encoded_models_predicted)
mse_normal = mean_squared_error(target_valid, model_predicted)
mse_encoded = mean_squared_error(target_valid, encoded_models_predicted)

print(f'Результат не закодированной матрицы: {r2score:.2f}; MSE: {mse_normal:.2f}')
print(f'Результат закодированной матрицы: {encoded_models_r2_score:.2f}; MSE: {mse_encoded:.2f}')
print(f'Result: {(r2score - encoded_models_r2_score):.2f}')
print(f'Метрики схожи? {np.isclose(r2score, encoded_models_r2_score)}')

Результат не закодированной матрицы: 0.42; MSE: 0.12
Результат закодированной матрицы: 0.42; MSE: 0.12
Result: -0.00
Метрики схожи? True


## Вывод по 4 шагу

*Предложенный алгоритм работает, закодировали данные. Обученные модели по исходным данным и закодироованным показали схожие метрики по R2 и MSE.*